In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")
import math
import os
import datetime
from pandas import DataFrame
import itertools
from bs4 import BeautifulSoup
import urllib.request
import requests
import random
import torch
import xmltodict
import xml.etree.ElementTree as ET
import xml
from xml.etree.ElementTree import parse
from xml.parsers import expat
from xml.dom.minidom import parse, parseString
import psycopg2
import PyPDF2
import pikepdf
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
import spacy

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joythompson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/joythompson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
username='joythompson'
directorybillhead="/Users/"+username+"/Desktop/DSE_203_JoyT/DSE_203_Final_Project_JoyT/Bill_Parsing_Headers/"
headers_v2=pd.read_csv(directorybillhead+ 'header_text_by_bill_header_with_order.csv')[0:50000].drop(columns=['Unnamed: 0', 'BillName', 'Header', 'HeaderOrderID', 'Key'])
headers_v2['Text']=headers_v2.groupby(['BillID', 'HeaderID'])['Text'].transform(lambda x : ' '.join(x))
header_gb=headers_v2.dropna(how='any').drop_duplicates()

# **Identify Unique & Categorized Nouns**

In [3]:
def findkeywordlabel(headers):
    uniquewordpd=pd.DataFrame(columns=['WordText','WordTextLower', 'CategoryLabel']);
    nlp = spacy.load("en_core_web_sm")
    wordlabelpd=pd.DataFrame(columns=['BillID', 'HeaderID', 'EntityOrderID', 'Text', 'CategoryLabel']);
    billidlist=[];
    for ind in headers.index:
        doc = nlp(str(headers.loc[ind,'Text']).replace('-', ' ').replace('     ', ''));
        entorder=0;
        for ent in doc.ents:
            entorder=entorder+1;
            if ent.text.find('Act')!=-1 or ent.text.find('ACT')!=-1:
                label='ACT';
            else:
                label=ent.label_;
            wordlabelpd=wordlabelpd.append({'BillID':headers.loc[ind, 'BillID'], 'HeaderID':headers.loc[ind, 'HeaderID'],
                                            'EntityOrderID':entorder , 'HeaderText': ent.text, 'CategoryLabel': label}, ignore_index=True);
            if ent.text.lower() not in uniquewordpd['WordTextLower'].values:
                uniquewordpd=uniquewordpd.append({'WordText': ent.text, 'WordTextLower': ent.text.lower(), 'CategoryLabel':label}, ignore_index=True);
        if str(headers.loc[ind, 'BillID'])[-1]=='0' and headers.loc[ind, 'BillID'] not in billidlist:
            billidlist.append(headers.loc[ind, 'BillID']);
    return wordlabelpd, uniquewordpd

In [4]:
noungroupspd_v2, uniquenounspd_v2=findkeywordlabel(header_gb)
noungroupspd_v2.to_csv(directorybillhead+ 'nouns_by_bill_header_with_groups_v2.csv')
uniquenounspd_v2.to_csv(directorybillhead+ 'unique_nouns_with_groups_v2.csv')

In [ ]:
wordlabelpd2, specialwordpd=findkeywordlabel(header_gb)
wordlabelpd2.to_csv(directorybillhead+ 'nouns_by_bill_header_with_groups_v3.csv')
specialwordpd.to_csv(directorybillhead+ 'unique_nouns_with_groups_v3.csv')

# **Deduplicate Unique & Categorized Nouns**

In [ ]:
alpha=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o', 'p', 'q', 'r','s','t','u','v',
       'w','x','y','z',',','-','_','–', ' ']
num=['0','1','2','3','4','5','6','7','8','9']
dropphraselist=['section', '(', 'usc', 'c. such']
dropcateglist=['ORDINAL', 'CARDINAL']
dropword1list=["the ","'", "–"]
dropword2list=["The ","", ""]

In [ ]:
def dropphrase(phrase, catpd):
    droplist=[]
    #Remove words/phrases containing the string phrase
    for ind in catpd.index:
        if phrase in catpd.loc[ind, 'WordTextLower'].replace('.',''):
            droplist.append(ind)
    catpd=catpd.drop(droplist).reset_index().drop(columns=['index'])
    return catpd

In [ ]:
def deduper(catpd,dropphraselist,dropcateglist, dropwordlist1):
    
    #Remove words/phrases containing the strings in dropphraselist
    for phrase in dropphraselist:
        catpd=dropphrase(phrase, catpd)
    
    #Remove words/phrases in the categories listed in dropcateglist
    for categ in dropcateglist: 
        catpd=catpd[catpd['CategoryLabel']!=categ]
        
    #Remove strings from words/phrases in word1list and word2list
    for word1 in dropword1list:
        for ind in catpd.index:
            catpd.loc[ind, 'WordTextLower']=catpd.loc[ind, 'WordTextLower'].replace(word1,'')
  
    #Remove words with over 95% similarity
    droplist=[]
    
    for ind in catpd.index:
        word=catpd.loc[ind, 'WordText']
        words=catpd.drop([ind])['WordText'].values
        ratio=process.extract(word, words)
        for match in ratio:
            if match[1] >95 and catpd[catpd['WordText']==match[0]].index[0] not in droplist:
            
                droplist.append(catpd[catpd['WordText']==match[0]].index[0])
    catpd=catpd.drop(droplist)
    return catpd

In [ ]:
def removespecial(catpd):   
    droplist=[]
    for ind in catpd.index:
        word=catpd.loc[ind, 'WordTextLower']
        wordcap=catpd.loc[ind, 'WordText']
        
        #Remove dates with non-numeric values and dates before 1776 and after 2050
        if catpd.loc[ind, 'CategoryLabel']=='DATE':
            alphayes=0
            for let in alpha:
                if let in word and ind not in droplist:
                    droplist.append(ind)
                    alphayes=1
            if alphayes==0:
                if float(word)>2050 or float(word)<1776:
                    droplist.append(ind)
        
        #Remove section number words (ex. 219c, 333jj)
        if len(word)>3:
            if len(word)<6 and word[0] in num and word[1] in num and word[2] in num and word[3] in alpha and ind not in droplist:
                droplist.append(ind)    
        
        #Remove leading spaces from word/phrase
        word=word.lstrip(' ') 
        wordcap=wordcap.lstrip(' ') 
    
    catpd=catpd.drop(droplist).reset_index().drop(columns=['index'])
    return catpd

In [ ]:
catpd=deduper(uniquenounspd_v2,dropphraselist,dropcateglist, dropword1list)
catpd=removespecial(catpd)
catpd.to_csv(directorybillhead+'ProcessedUniqueNounCategory.csv')

# **Complete Sentence/Phrase POS Breakdown Using Unique Nouns**

In [ ]:
from collections.abc import Iterable
# dependency markers for subjects
SUBJECTS = {"nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"}
# dependency markers for objects
OBJECTS = {"dobj", "dative", "attr", "oprd"}
# POS tags that will break adjoining items
BREAKER_POS = {"CCONJ", "VERB"}
# words that are negations
NEGATIONS = {"no", "not", "n't", "never", "none"}


# does dependency set contain any coordinating conjunctions?
def contains_conj(depSet):
    return "and" in depSet or "or" in depSet or "nor" in depSet or \
           "but" in depSet or "yet" in depSet or "so" in depSet or "for" in depSet


# get subs joined by conjunctions
def _get_subs_from_conjunctions(subs):
    more_subs = []
    for sub in subs:
        # rights is a generator
        rights = list(sub.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if contains_conj(rightDeps):
            more_subs.extend([tok for tok in rights if tok.dep_ in SUBJECTS or tok.pos_ == "NOUN"])
            if len(more_subs) > 0:
                more_subs.extend(_get_subs_from_conjunctions(more_subs))
    return more_subs


# get objects joined by conjunctions
def _get_objs_from_conjunctions(objs):
    more_objs = []
    for obj in objs:
        # rights is a generator
        rights = list(obj.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if contains_conj(rightDeps):
            more_objs.extend([tok for tok in rights if tok.dep_ in OBJECTS or tok.pos_ == "NOUN"])
            if len(more_objs) > 0:
                more_objs.extend(_get_objs_from_conjunctions(more_objs))
    return more_objs


# find sub dependencies
def _find_subs(tok):
    head = tok.head
    while head.pos_ != "VERB" and head.pos_ != "NOUN" and head.head != head:
        head = head.head
    if head.pos_ == "VERB":
        subs = [tok for tok in head.lefts if tok.dep_ == "SUB"]
        if len(subs) > 0:
            verb_negated = _is_negated(head)
            subs.extend(_get_subs_from_conjunctions(subs))
            return subs, verb_negated
        elif head.head != head:
            return _find_subs(head)
    elif head.pos_ == "NOUN":
        return [head], _is_negated(tok)
    return [], False


# is the tok set's left or right negated?
def _is_negated(tok):
    parts = list(tok.lefts) + list(tok.rights)
    for dep in parts:
        if dep.lower_ in NEGATIONS:
            return True
    return False


# get all the verbs on tokens with negation marker
def _find_svs(tokens):
    svs = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB"]
    for v in verbs:
        subs, verbNegated = _get_all_subs(v)
        if len(subs) > 0:
            for sub in subs:
                svs.append((sub.orth_, "!" + v.orth_ if verbNegated else v.orth_))
    return svs


# get grammatical objects for a given set of dependencies (including passive sentences)
def _get_objs_from_prepositions(deps, is_pas):
    objs = []
    for dep in deps:
        if dep.pos_ == "ADP" and (dep.dep_ == "prep" or (is_pas and dep.dep_ == "agent")):
            objs.extend([tok for tok in dep.rights if tok.dep_  in OBJECTS or
                         (tok.pos_ == "PRON" and tok.lower_ == "me") or
                         (is_pas and tok.dep_ == 'pobj')])
    return objs


# get objects from the dependencies using the attribute dependency
def _get_objs_from_attrs(deps, is_pas):
    for dep in deps:
        if dep.pos_ == "NOUN" and dep.dep_ == "attr":
            verbs = [tok for tok in dep.rights if tok.pos_ == "VERB"]
            if len(verbs) > 0:
                for v in verbs:
                    rights = list(v.rights)
                    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
                    objs.extend(_get_objs_from_prepositions(rights, is_pas))
                    if len(objs) > 0:
                        return v, objs
    return None, None


# xcomp; open complement - verb has no suject
def _get_obj_from_xcomp(deps, is_pas):
    for dep in deps:
        if dep.pos_ == "VERB" and dep.dep_ == "xcomp":
            v = dep
            rights = list(v.rights)
            objs = [tok for tok in rights if tok.dep_ in OBJECTS]
            objs.extend(_get_objs_from_prepositions(rights, is_pas))
            if len(objs) > 0:
                return v, objs
    return None, None


# get all functional subjects adjacent to the verb passed in
def _get_all_subs(v):
    verb_negated = _is_negated(v)
    subs = [tok for tok in v.lefts if tok.dep_ in SUBJECTS and tok.pos_ != "DET"]
    if len(subs) > 0:
        subs.extend(_get_subs_from_conjunctions(subs))
    else:
        foundSubs, verb_negated = _find_subs(v)
        subs.extend(foundSubs)
    return subs, verb_negated


# find the main verb - or any aux verb if we can't find it
def _find_verbs(tokens):
    verbs = [tok for tok in tokens if _is_non_aux_verb(tok)]
    if len(verbs) == 0:
        verbs = [tok for tok in tokens if _is_verb(tok)]
    return verbs


# is the token a verb?  (excluding auxiliary verbs)
def _is_non_aux_verb(tok):
    return tok.pos_ == "VERB" and (tok.dep_ != "aux" and tok.dep_ != "auxpass")


# is the token a verb?  (excluding auxiliary verbs)
def _is_verb(tok):
    return tok.pos_ == "VERB" or tok.pos_ == "AUX"


# return the verb to the right of this verb in a CCONJ relationship if applicable
# returns a tuple, first part True|False and second part the modified verb if True
def _right_of_verb_is_conj_verb(v):
    # rights is a generator
    rights = list(v.rights)

    # VERB CCONJ VERB (e.g. he beat and hurt me)
    if len(rights) > 1 and rights[0].pos_ == 'CCONJ':
        for tok in rights[1:]:
            if _is_non_aux_verb(tok):
                return True, tok

    return False, v


# get all objects for an active/passive sentence
def _get_all_objs(v, is_pas):
    # rights is a generator
    rights = list(v.rights)

    objs = [tok for tok in rights if tok.dep_ in OBJECTS or (is_pas and tok.dep_ == 'pobj')]
    objs.extend(_get_objs_from_prepositions(rights, is_pas))

    #potentialNewVerb, potentialNewObjs = _get_objs_from_attrs(rights)
    #if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
    #    objs.extend(potentialNewObjs)
    #    v = potentialNewVerb

    potential_new_verb, potential_new_objs = _get_obj_from_xcomp(rights, is_pas)
    if potential_new_verb is not None and potential_new_objs is not None and len(potential_new_objs) > 0:
        objs.extend(potential_new_objs)
        v = potential_new_verb
    if len(objs) > 0:
        objs.extend(_get_objs_from_conjunctions(objs))
    return v, objs


# return true if the sentence is passive - at he moment a sentence is assumed passive if it has an auxpass verb
def _is_passive(tokens):
    for tok in tokens:
        if tok.dep_ == "auxpass":
            return True
    return False


# resolve a 'that' where/if appropriate
def _get_that_resolution(toks):
    for tok in toks:
        if 'that' in [t.orth_ for t in tok.lefts]:
            return tok.head
    return None


# simple stemmer using lemmas
def _get_lemma(word: str):
    tokens = nlp(word)
    if len(tokens) == 1:
        return tokens[0].lemma_
    return word


# print information for displaying all kinds of things of the parse tree
def printDeps(toks):
    for tok in toks:
        #print(tok.orth_, tok.dep_, tok.pos_, tok.head.orth_, [t.orth_ for t in tok.lefts], [t.orth_ for t in tok.rights])


# expand an obj / subj np using its chunk
def expand(item, tokens, visited):
    if item.lower_ == 'that':
        temp_item = _get_that_resolution(tokens)
        if temp_item is not None:
            item = temp_item

    parts = []

    if hasattr(item, 'lefts'):
        for part in item.lefts:
            if part.pos_ in BREAKER_POS:
                break
            if not part.lower_ in NEGATIONS:
                parts.append(part)

    parts.append(item)

    if hasattr(item, 'rights'):
        for part in item.rights:
            if part.pos_ in BREAKER_POS:
                break
            if not part.lower_ in NEGATIONS:
                parts.append(part)

    if hasattr(parts[-1], 'rights'):
        for item2 in parts[-1].rights:
            if item2.pos_ == "DET" or item2.pos_ == "NOUN":
                if item2.i not in visited:
                    visited.add(item2.i)
                    parts.extend(expand(item2, tokens, visited))
            break

    return parts


# convert a list of tokens to a string
def to_str(tokens):
    if isinstance(tokens, Iterable):
        return ' '.join([item.text for item in tokens])
    else:
        return ''


# find verbs and their subjects / objects to create SVOs, detect passive/active sentences
def findSVOs(tokens):
    svos = []
    is_pas = _is_passive(tokens)
    verbs = _find_verbs(tokens)
    visited = set()  # recursion detection
    for v in verbs:
        subs, verbNegated = _get_all_subs(v)
        # hopefully there are subs, if not, don't examine this verb any longer
        if len(subs) > 0:
            isConjVerb, conjV = _right_of_verb_is_conj_verb(v)
            if isConjVerb:
                v2, objs = _get_all_objs(conjV, is_pas)
                for sub in subs:
                    for obj in objs:
                        objNegated = _is_negated(obj)
                        if is_pas:  # reverse object / subject for passive
                            svos.append((to_str(expand(obj, tokens, visited)),
                                         "!" + v.lemma_ if verbNegated or objNegated else v.lemma_, to_str(expand(sub, tokens, visited))))
                            svos.append((to_str(expand(obj, tokens, visited)),
                                         "!" + v2.lemma_ if verbNegated or objNegated else v2.lemma_, to_str(expand(sub, tokens, visited))))
                        else:
                            svos.append((to_str(expand(sub, tokens, visited)),
                                         "!" + v.lower_ if verbNegated or objNegated else v.lower_, to_str(expand(obj, tokens, visited))))
                            svos.append((to_str(expand(sub, tokens, visited)),
                                         "!" + v2.lower_ if verbNegated or objNegated else v2.lower_, to_str(expand(obj, tokens, visited))))
            else:
                v, objs = _get_all_objs(v, is_pas)
                for sub in subs:
                    if len(objs) > 0:
                        for obj in objs:
                            objNegated = _is_negated(obj)
                            if is_pas:  # reverse object / subject for passive
                                svos.append((to_str(expand(obj, tokens, visited)),
                                             "!" + v.lemma_ if verbNegated or objNegated else v.lemma_, to_str(expand(sub, tokens, visited))))
                            else:
                                svos.append((to_str(expand(sub, tokens, visited)),
                                             "!" + v.lower_ if verbNegated or objNegated else v.lower_, to_str(expand(obj, tokens, visited))))
                    else:
                        # no obj - just return the SV parts
                        svos.append((to_str(expand(sub, tokens, visited)),
                                     "!" + v.lower_ if verbNegated else v.lower_,))

    return svos

In [ ]:
def allwordPOS2(header_gb):
    allpd=pd.DataFrame(columns=['BillID', 'HeaderID', 'EntityOrderID', 'Text', 'Sub', 'Verb', 'Pred'])
    nlp= spacy.load("en_core_web_sm")
    for ind in header_gb.index:
        doc = nlp(header_gb.loc[ind,'Text']) 
        svo=findSVOs(doc)
        if len(svo)>0:
            if len(svo[0])>1:
                if len(svo[0])<3:
                    pred='NA'
                else:
                    pred=svo[0][2] 

                allpd=allpd.append({ 'BillID': header_gb.loc[ind, 'BillID'], 
                                    'HeaderID': header_gb.loc[ind, 'HeaderID'], 'Text':doc,
                                    'Sub':svo[0][0], 'Verb': svo[0][1], 'Pred':pred},ignore_index=True)
    return allpd

In [ ]:
def allwordPOS3(allpd):
    droplist=[]
    nlp= spacy.load("en_core_web_sm")
    allpd['Sub Category']=''
    allpd['Pred Category']=''
    for ind in allpd.index:
        sub=0
        pred=0
        for wordind in catpd.index:
            word=catpd.loc[wordind,'WordTextLower']
            if word in allpd.loc[ind,'Sub'].lower():
                allpd.loc[ind,'Sub']=word
                allpd.loc[ind,'Sub Category']=catpd.loc[wordind,'CategoryLabel']
                sub=sub+1
            if word in allpd.loc[ind,'Pred'].lower():
                allpd.loc[ind,'Pred']=word
                allpd.loc[ind,'Pred Category']=catpd.loc[wordind,'CategoryLabel']
                pred=pred+1
        if len(allpd.loc[ind,'Sub Category'])==0 or len(allpd.loc[ind,'Pred Category'])==0:
            droplist.append(ind)
    allpd2=allpd.drop(droplist)      
    return allpd2

In [ ]:
def filterallpd(allpd2):
    droplist=[]
   
    for ind in allpd2.index:
       
        if allpd2.loc[ind,'Sub'].lower()=='na' or allpd2.loc[ind,'Pred'].lower()=='na':
            droplist.append(ind)
        elif '(' in allpd2.loc[ind,'Sub'].lower() or '(' in allpd2.loc[ind,'Pred'].lower():
            droplist.append(ind)
        elif ')' in allpd2.loc[ind,'Sub'].lower() or ')' in allpd2.loc[ind,'Pred'].lower(): 
            droplist.append(ind)
        elif allpd2.loc[ind,'Verb'].lower()=='is':
            droplist.append(ind)
        elif 'usc' in allpd2.loc[ind,'Sub'].lower().replace('.','') or 'usc' in allpd2.loc[ind,'Pred'].lower().replace('.',''): 
            droplist.append(ind)
        
        allpd2.loc[ind, 'Sub']=allpd.loc[ind, 'Sub'].lower().lstrip( 'a ')
        allpd2.loc[ind, 'Pred']=allpd.loc[ind, 'Pred'].lower().lstrip( 'a ')  
        allpd2.loc[ind, 'Sub']=allpd.loc[ind, 'Sub'].lower().replace('the', '').replace('of','').replace(' a ', '')
        allpd2.loc[ind, 'Pred']=allpd.loc[ind, 'Pred'].lower().replace('the', '').replace('of','').replace(' a ', '')
        allpd2.loc[ind, 'Verb']=allpd.loc[ind, 'Verb'].lower().replace('the', '').replace('of','').replace(' a ', '')
    allpd3=allpd2.drop(droplist) 
    return allpd3

In [ ]:
allpd=allwordPOS2(header_gb)
allpd=allpd.reset_index().drop(columns=['index'])
allpd2=allwordPOS3(allpd)
allpd2=allpd2.reset_index().drop(columns=['index'])
allpd3=filterallpd(allpd2)
allpd3=allpd3.reset_index().drop(columns=['index'])

In [ ]:
allpd3.to_csv(directorybillhead+'allpd3.csv')

In [ ]:
print(allpd3)

In [ ]:
def allwordPOS(header_gb):
    verbspd=pd.DataFrame(columns=['BillID', 'HeaderID', 'EntityOrderID', 'Text','PartofSpeech', 'SubjObjPred', 'HeadText'])
    nlp= spacy.load("en_core_web_sm")
    for ind in header_gb.index:
        doc = nlp(header_gb.loc[ind,'Text']) 
        span = doc[doc[len(doc)-1].left_edge.i : doc[len(doc)-1].right_edge.i+1]
        with doc.retokenize() as retokenizer:
            retokenizer.merge(span)
            entorder=0 
            for token in doc:
                entorder=entorder+1
                verbspd=verbspd.append({ 'BillID': header_gb.loc[ind, 'BillID'], 
                            'HeaderID': header_gb.loc[ind, 'HeaderID'],'EntityOrderID':entorder, 'Text':token.text,
                            'PartofSpeech':token.pos_, 'SubjObjPred': token.dep_, 'HeadText':token.head.text},ignore_index=True)
    return verbspd